In [ ]:
#Nama:Fawwaz Anugrah Wiradhika Dharmasatya
#NIM:16520059
#Universitas:Institut Teknologi Bandung(ITB)
#Tanggal mulai:15 November 2020
#Sumber dataset:respons_covid.csv

#Problem Framing
#Menganalisis data kegiatan dan risiko terpapar covid dan menampilkan hasil analisisnya

#Data Cleaning

In [163]:
import pandas as pd
from sklearn.neighbors import NearestNeighbors
import pickle
import numpy as np

In [150]:
df = pd.read_csv("/content/respons_covid.csv")
df.head()

,1. Apakah anda pernah bertemu pasien positif Covid-19 (berada dalam satu ruangan yang sama/kontak dalam jarak 1 meter)?,2. Apakah Anda tinggal di zona merah?,3. Apakah Anda pernah melakukan perjalanan ke luar negri?,4. Seberapa sering Anda pergi ke luar rumah?,5. Jarak terjauh yang Anda tempuh ketika bepergian?,"6. Apakah Anda berpergian kemana-mana menggunakan transportasi umum (transportasi online, angkot, bus, taksi, kereta)?",7. Apakah Anda memakai masker setiap kali bertemu dengan orang lain?,8. Jenis masker apa yang biasa Anda gunakan?,9. Apakah Anda berjabat tangan dengan orang lain?,10. Apakah Anda menyentuh benda yang juga disentuh oleh orang lain?,11. Dengan apa Anda biasa membersihkan tangan?,12. Apakah Anda segera mandi dan keramas setelah berpergian dari luar rumah?,"13. Apakah Anda menjaga jarak 1,5 meter dengan orang lain ketika belanja, bekerja atau pergi ke tempat ibadah?","14. Seberapa sering Anda makan di luar rumah (warung, cafe atau restoran)?",15. Apakah Anda membeli makanan dari luar rumah untuk dibawa pulang?,16. Apakah Anda memanaskan makanan yang dibeli di luar terlebih dahulu sebelum dikonsumsi?,17. Apakah Anda membersihkan/menjemur terlebih dahulu paket yang Anda terima?,18. Apakah Anda tinggal di lingkungan padat penduduk?,19. Bagaimana kondisi tempat tinggal Anda,20. Bagaimana aktivitas Anda sehari-hari?,21. Apakah Anda terkena cahaya matahari minimal 15 menit (berjemur)?,22. Apakah Anda berjalan kaki/berolahraga minimal 30 menit setiap hari?,23. Apakah Anda mengonsumsi vitamin/suplemen penambah imun?,24. Berapa lamakah waktu tidur Anda?,25. Berapakah usia Anda?,26. Apakah ada riwayat penyakit yang Anda miliki?
0,Tidak/Tidak Tahu,Ya,Tidak,1-3 kali dalam seminggu,Luar kota,Tidak,Ya,Masker bedah,Ya,Ya,Air mengalir + sabun,Tidak,Tidak,Kadang-kadang,Ya,Ya,Tidak,Tidak,Ruangan tertutup dengan AC,Di luar rumah dengan intensitas bertemu orang ...,Tidak,Ya,Ya,3-4 jam,19-45 tahun,Tidak ada/Selain yang di atas
1,Tidak/Tidak Tahu,Ya,Tidak,1-3 kali dalam seminggu,Kurang dari 500 meter,Tidak,Ya,Masker bedah,Tidak,Tidak,Air mengalir + sabun,Ya,Ya,Tidak pernah,Tidak,Tidak,Ya,Tidak,Ruangan dengan sirkulasi udara yang baik,Di dalam rumah/WFH,Ya,Tidak,Ya,7-8 jam,19-45 tahun,Tidak ada/Selain yang di atas
2,Ya,Ya,Tidak,Tidak pernah,Kurang dari 500 meter,Ya,Ya,Masker bedah,Tidak,Ya,Air mengalir + sabun,Ya,Ya,Kadang-kadang,Ya,Tidak,Ya,Ya,Ruangan dengan sirkulasi udara yang baik,Di dalam rumah/WFH,Ya,Tidak,Ya,5-6 jam,19-45 tahun,Tidak ada/Selain yang di atas
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Tidak/Tidak Tahu,Ya,Tidak,4-6 kali dalam seminggu,Luar provinsi,Ya,Ya,Masker kain,Tidak,Ya,Hand sanitizer beralkohol minimal 70%,Ya,Ya,Kadang-kadang,Tidak,Tidak,Ya,Ya,Ruangan dengan sirkulasi udara yang baik,Di dalam rumah/WFH,Tidak,Tidak,Tidak,5-6 jam,19-45 tahun,Tidak ada/Selain yang di atas


In [151]:
df.columns = ['contact','red_zone','oversea','frec_go','farthest_dist','public_transport','mask','mask_type','handshake','touch','wash_hand','take_bath','social_distancing','eat_outside_frec','eat_outside','heat_first','jemur_paket','crowded_neighborhood','neighborhood_condition','daily_activity','sunbathing','sport_jogging','eat_vitamin','sleep_time','age_range','disease_track']

In [ ]:
#Nama kolomnya diperpendek biar gampang

In [152]:
df.head()

,contact,red_zone,oversea,frec_go,farthest_dist,public_transport,mask,mask_type,handshake,touch,wash_hand,take_bath,social_distancing,eat_outside_frec,eat_outside,heat_first,jemur_paket,crowded_neighborhood,neighborhood_condition,daily_activity,sunbathing,sport_jogging,eat_vitamin,sleep_time,age_range,disease_track
0,Tidak/Tidak Tahu,Ya,Tidak,1-3 kali dalam seminggu,Luar kota,Tidak,Ya,Masker bedah,Ya,Ya,Air mengalir + sabun,Tidak,Tidak,Kadang-kadang,Ya,Ya,Tidak,Tidak,Ruangan tertutup dengan AC,Di luar rumah dengan intensitas bertemu orang ...,Tidak,Ya,Ya,3-4 jam,19-45 tahun,Tidak ada/Selain yang di atas
1,Tidak/Tidak Tahu,Ya,Tidak,1-3 kali dalam seminggu,Kurang dari 500 meter,Tidak,Ya,Masker bedah,Tidak,Tidak,Air mengalir + sabun,Ya,Ya,Tidak pernah,Tidak,Tidak,Ya,Tidak,Ruangan dengan sirkulasi udara yang baik,Di dalam rumah/WFH,Ya,Tidak,Ya,7-8 jam,19-45 tahun,Tidak ada/Selain yang di atas
2,Ya,Ya,Tidak,Tidak pernah,Kurang dari 500 meter,Ya,Ya,Masker bedah,Tidak,Ya,Air mengalir + sabun,Ya,Ya,Kadang-kadang,Ya,Tidak,Ya,Ya,Ruangan dengan sirkulasi udara yang baik,Di dalam rumah/WFH,Ya,Tidak,Ya,5-6 jam,19-45 tahun,Tidak ada/Selain yang di atas
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Tidak/Tidak Tahu,Ya,Tidak,4-6 kali dalam seminggu,Luar provinsi,Ya,Ya,Masker kain,Tidak,Ya,Hand sanitizer beralkohol minimal 70%,Ya,Ya,Kadang-kadang,Tidak,Tidak,Ya,Ya,Ruangan dengan sirkulasi udara yang baik,Di dalam rumah/WFH,Tidak,Tidak,Tidak,5-6 jam,19-45 tahun,Tidak ada/Selain yang di atas


#Data Preprocessing

In [153]:
#Konversi string jadi angka biar gampang

In [154]:
df['contact']= df['contact'].apply(lambda x:1 if x=='Ya' else 0)
df['red_zone']= df['red_zone'].apply(lambda x:1 if x=='Ya' else 0)
df['oversea']= df['oversea'].apply(lambda x:1 if x=='Ya' else 0)
df['frec_go']= df['frec_go'].apply(lambda x:4 if x=='Setiap hari' else (3 if x=='4-6 kali dalam seminggu' else (2 if x=='1-3 kali dalam seminggu' else (1 if x=='Kurang dari seminggu sekali' else 0))))
df['farthest_dist']= df['farthest_dist'].apply(lambda x:4 if x=='Luar negri' else (3 if x=='Luar pulau' else (2 if x=='Luar provinsi' else (1 if x=='Luar kota' else 0))))
df['public_transport']= df['public_transport'].apply(lambda x:1 if x=='Ya' else 0)
df['mask']= df['mask'].apply(lambda x:0 if x=='Ya' else 1)
df['mask_type']= df['mask_type'].apply(lambda x:0 if x=='Masker bedah' else (1 if x=='Masker N95' else (2 if x=='Masker kain' else (3 if x=='Tidak pakai masker' else 4))))
df['handshake']= df['handshake'].apply(lambda x:1 if x=='Ya' else 0)
df['touch']= df['touch'].apply(lambda x:1 if x=='Ya' else 0)
df['wash_hand']= df['wash_hand'].apply(lambda x:0 if x=='Air mengalir + sabun' else (1 if x=='Hand sanitizer beralkohol minimal 70%' else (2 if x=='Tisu basah beralkohol minimal 70%' else 3)))
df['take_bath']= df['take_bath'].apply(lambda x:0 if x=='Ya' else 1)
df['social_distancing']= df['social_distancing'].apply(lambda x:0 if x=='Ya' else 1)
df['eat_outside_frec']= df['eat_outside_frec'].apply(lambda x:2 if x=='Setiap saat' else (1 if x=='Kadang-kadang' else 0))
df['eat_outside']= df['eat_outside'].apply(lambda x:1 if x=='Ya' else 0)
df['heat_first']= df['heat_first'].apply(lambda x:0 if x=='Ya' else 1)
df['jemur_paket']= df['jemur_paket'].apply(lambda x:0 if x=='Ya' else 1)
df['crowded_neighborhood']= df['crowded_neighborhood'].apply(lambda x:1 if x=='Ya' else 0)
df['neighborhood_condition']= df['neighborhood_condition'].apply(lambda x:1 if x=='Ruangan tertutup dengan AC' else 0)
df['daily_activity']= df['daily_activity'].apply(lambda x:2 if x=='Di luar rumah dengan intensitas bertemu orang lain yang tinggi' else (1 if x=='Di luar rumah dengan intensitas bertemu orang lain yang rendah' else 0))
df['sunbathing']= df['sunbathing'].apply(lambda x:0 if x=='Ya' else 1)
df['sport_jogging']= df['sport_jogging'].apply(lambda x:0 if x=='Ya' else 1)
df['eat_vitamin']= df['eat_vitamin'].apply(lambda x:0 if x=='Ya' else 1)
df['sleep_time']= df['sleep_time'].apply(lambda x:4 if x=='Kurang dari 3 jam' else (3 if x=='3-4 jam' else (2 if x=='5-6 jam' else (1 if x=='7-8 jam' else 0))))
df['age_range']= df['age_range'].apply(lambda x:0 if x=='Di bawah 19 tahun' else (1 if x=='19-45 tahun' else (2 if x=='46-65 tahun' else 3)))
df['disease_track']= df['disease_track'].apply(lambda x:5 if x=='Gangguan pernapasan kronis' else (4 if x=='Penyakit kardiovaskular ( penyakit jantung koroner, gagal jantung, stroke, dan hipertensi)' else (3 if x=='Diabetes' else (2 if x=='Kanker' else (1 if x=='Penyakit ginjal' else 0)))))

In [155]:
df.head()

,contact,red_zone,oversea,frec_go,farthest_dist,public_transport,mask,mask_type,handshake,touch,wash_hand,take_bath,social_distancing,eat_outside_frec,eat_outside,heat_first,jemur_paket,crowded_neighborhood,neighborhood_condition,daily_activity,sunbathing,sport_jogging,eat_vitamin,sleep_time,age_range,disease_track
0,0,1,0,2,1,0,0,0,1,1,0,1,1,1,1,0,1,0,1,1,1,0,0,3,1,0
1,0,1,0,2,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,1,1,0
2,1,1,0,0,0,1,0,0,0,1,0,0,0,1,1,1,0,1,0,0,0,1,0,2,1,0
3,0,0,0,0,0,0,1,4,0,0,3,1,1,0,0,1,1,0,0,0,1,1,1,0,3,0
4,0,1,0,3,2,1,0,2,0,1,1,0,0,1,0,1,0,1,0,0,1,1,1,2,1,0


#Model Development

In [156]:
df['sum'] = df.sum(axis=1) #Membuat kolom untuk menghitung jumlah nilai tiap orang

In [157]:
df.head()

,contact,red_zone,oversea,frec_go,farthest_dist,public_transport,mask,mask_type,handshake,touch,wash_hand,take_bath,social_distancing,eat_outside_frec,eat_outside,heat_first,jemur_paket,crowded_neighborhood,neighborhood_condition,daily_activity,sunbathing,sport_jogging,eat_vitamin,sleep_time,age_range,disease_track,sum
0,0,1,0,2,1,0,0,0,1,1,0,1,1,1,1,0,1,0,1,1,1,0,0,3,1,0,18
1,0,1,0,2,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,1,1,0,7
2,1,1,0,0,0,1,0,0,0,1,0,0,0,1,1,1,0,1,0,0,0,1,0,2,1,0,12
3,0,0,0,0,0,0,1,4,0,0,3,1,1,0,0,1,1,0,0,0,1,1,1,0,3,0,18
4,0,1,0,3,2,1,0,2,0,1,1,0,0,1,0,1,0,1,0,0,1,1,1,2,1,0,20


In [ ]:
#Nilai skor maksnya 48
#0-15 Risiko rendah
#16-31 Risiko sedang
#32-48 Risiko tinggi

In [164]:
conditions = [
    (df['sum'] >=0) & (df['sum'] <16),
    (df['sum'] >=16) & (df['sum'] <32),
    (df['sum'] >=32)]
risiko = ['Rendah', 'Sedang', 'Tinggi']
df['risk'] = np.select(conditions, risiko)

In [165]:
df.head()

,contact,red_zone,oversea,frec_go,farthest_dist,public_transport,mask,mask_type,handshake,touch,wash_hand,take_bath,social_distancing,eat_outside_frec,eat_outside,heat_first,jemur_paket,crowded_neighborhood,neighborhood_condition,daily_activity,sunbathing,sport_jogging,eat_vitamin,sleep_time,age_range,disease_track,sum,risk
0,0,1,0,2,1,0,0,0,1,1,0,1,1,1,1,0,1,0,1,1,1,0,0,3,1,0,18,Sedang
1,0,1,0,2,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,1,1,0,7,Rendah
2,1,1,0,0,0,1,0,0,0,1,0,0,0,1,1,1,0,1,0,0,0,1,0,2,1,0,12,Rendah
3,0,0,0,0,0,0,1,4,0,0,3,1,1,0,0,1,1,0,0,0,1,1,1,0,3,0,18,Sedang
4,0,1,0,3,2,1,0,2,0,1,1,0,0,1,0,1,0,1,0,0,1,1,1,2,1,0,20,Sedang


#Presentation

In [ ]:
#Mencari orang dengan risiko terendah

In [166]:
df['sum'].idxmin()

1

In [168]:
print(df['sum'][1])

7


In [ ]:
#Nilai risiko terkecil adalah 7 yg diperoleh orang kedua(karena indeksnya dimulai dari 0)

In [ ]:
#Mencari orang dengan risiko tertinggi

In [169]:
df['sum'].idxmax()

14

In [170]:
print(df['sum'][14])

25


In [ ]:
#Nilai risiko terbesar adalah 25 yg diperoleh orang ke-15(karena indeksnya dimulai dari 0)

In [ ]:
#Rata-rata skor akhir

In [171]:
df['sum'].mean()

17.38235294117647

In [ ]:
#Rata-rata skor respondens adalah 17.38 yang masuk risiko sedang

In [ ]:
#Insight baru

In [ ]:
#1.Semakin tinggi nilai sum,semakin berisiko terpapar COVID-19

In [ ]:
#2.Kebanyakan respondens memiliki risiko sedang terpapar Covid-19

#Model Deployment

In [172]:
pickle.dump(df,open('/content/analisis_risiko.pickle','wb'))

In [173]:
df.to_csv("/content/hasil_analisis.csv",index=False)

In [ ]:
#Hasil analisis berupa 2 kolom terakhir pada tabel di hasil_analisis.csv